In [ ]:
file = open('./IMDb-data/positive-samples.pickle', 'rb')
train_pos = pickle.load(file)
file.close()

In [ ]:
file = open('./IMDb-data/pos-train.pickle', 'rb')
train_pos_labels = pickle.load(file)
file.close()

In [ ]:
file = open('./IMDb-data/negative-samples.pickle', 'rb')
train_neg = pickle.load(file)
file.close()

In [ ]:
file = open('./IMDb-data/neg-train.pickle', 'rb')
train_neg_labels = pickle.load(file)
file.close()

In [ ]:
train = train_pos + train_neg

In [ ]:
train[12]

In [ ]:
train_labels = train_pos_labels + train_neg_labels

In [ ]:
train_labels[12]

In [ ]:
train[12]

In [ ]:
def clean_binput(text):
    return text.decode().replace('<br />', '').replace('.', ' ').replace('(', '').replace(')', '').replace(',', '')

In [ ]:
def numerify(ins, word_list):
    vectorized = []
    for word in ins:
        if word in word_list:
            vectorized.append(word_list.index(word)+1)
    return vectorized

In [ ]:
def extract_label(label_fname):
     return int(label_fname.split('_')[-1].split('.')[0])

In [ ]:
trainset = []
for i in train:
    x = clean_binput(i).split()
    trainset.append(numerify(x, words))
    

In [ ]:
trainlabels = []
for i in train_labels:
    trainlabels.append(extract_label(i))

In [ ]:
for i in range(len(trainset)):
    if len(trainset[i]) < 2000:
        while len(trainset[i]) < 2000:
            trainset[i].insert(0, 0)
    

In [ ]:
inputs = T.tensor(trainset)

In [ ]:
inputs = inputs.reshape((25000, 80, 25))

In [ ]:
inputs = inputs.to(T.float)

In [ ]:
labels = trainlabels

In [ ]:
f = h5py.File('train.h5py', 'w')
f.create_dataset('train_data', data=inputs)
f.create_dataset('train_labels', data=labels)
f.close()

In [ ]:
labels

In [ ]:
file = open('trainset', 'wb')
pickle.dump(trainset, file)
file.close()

In [ ]:
file = open('train', 'wb')
pickle.dump(train, file)
file.close()

In [ ]:
file = open('trainset', 'rb')
trainset = pickle.load(file)
file.close()

In [ ]:
file = open('trainlabels', 'rb')
trainlabels = pickle.load(file)
file.close()

In [ ]:
len(maxItem)

In [ ]:
inputs.shape

In [ ]:
inputs[0]

In [ ]:
import numpy as np

In [ ]:
len(train)

In [ ]:
splits = list(zip(*train)) 

In [ ]:
test_format = list(splits[0])

In [ ]:
test_format[0].shape

In [ ]:
test_format[0].shape

In [ ]:
for i in range(len(test_format)):
    test_format[i] = test_format[i].reshape((80,25))

In [ ]:
list(splits[1])

In [ ]:
train[0]

In [ ]:
train[0][0].reshape((2000,1))

In [ ]:
embedder(T.tensor(train[0][0]).to(T.long))

In [ ]:
output[:,-1]

In [ ]:
memory

# Load data and start from here

In [1]:
import torch as T
from dnc import DNC

In [2]:
import pickle
import h5py

In [3]:
file = open('./IMDb-data/imdbwords.pickle', 'rb')
words = pickle.load(file)
file.close()

In [4]:
f = h5py.File('train.h5py', 'r+')
trains = f.get('train_data')
trainlabels = f.get('train_labels')
for i in range(len(trainlabels)):
    trainlabels[i] = 1 if i < 12500 else 0
trainz = zip(trains, trainlabels)
train = list(trainz)
f.close()

In [5]:
f = h5py.File('test.h5py', 'r+')
tests = f.get('test_data')
testlabels = f.get('test_labels')
for i in range(len(testlabels)):
    testlabels[i] = 1 if i < 12500 else 0
testz = zip(tests, testlabels)
test = list(testz)
f.close()

In [6]:
in_size =  25 #5 with embed

In [37]:
diffy = DNC(input_size=in_size, hidden_size=28, num_layers=2, independent_linears=True)

In [ ]:
embedder = T.nn.Embedding(len(words)+1, in_size)

In [8]:
loss_fn = T.nn.MSELoss() #T.nn.MSELoss() 

first epoch lr 0.01

In [ ]:
import itertools
params = diffy.parameters() #[embedder.parameters(), diffy.parameters()]

In [45]:
optimizer = T.optim.Adam(diffy.parameters(), lr=1e-5, eps=1e-9, betas=[0.9, 0.98]) #itertools.chain(*params)

In [39]:
(controller_hidden, memory, read_vectors) = (None, None, None)

In [55]:
batch_size = 50
train_data_loader = T.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
trainload = iter(train_data_loader)

In [56]:
ranges = len(trainload)

In [ ]:
diffy.train()
#embedder.train()

for it in range(ranges):
    optimizer.zero_grad()
    
    ins, tru = next(trainload)
    
    #Forward pass
    #embed = embedder(ins.reshape(batch_size, 2000).to(T.long))
    output, (controller_hidden, memory, read_vectors) = diffy(ins, (None, memory, None), reset_experience=False)
    
    #output_2 = output[:,-1]
    final_out = T.sum(output, (1, 2), keepdim=True) #outer(mid_out)
    #final_out = T.sum(T.sum(output, dim=1, keepdim=True), dim=2, keepdim=True)
    
    #final_out = T.nn.functional.sigmoid(final_out)

    loss = loss_fn(final_out, tru.to(T.float).reshape((batch_size,1,1)))
    
    loss.backward()
    T.nn.utils.clip_grad_value_(diffy.parameters(), 10)
    optimizer.step()
    
    memory = {k : (v.detach() if isinstance(v, T.autograd.Variable) else v) for k, v in memory.items()}
    
    print('Step: {}, Loss: {}'.format(it+1, loss.item()))
            
    

Step: 1, Loss: 0.2978907823562622
Step: 2, Loss: 0.29356178641319275
Step: 3, Loss: 0.287936270236969
Step: 4, Loss: 0.28123942017555237
Step: 5, Loss: 0.268457293510437
Step: 6, Loss: 0.2568327784538269
Step: 7, Loss: 0.2680361866950989
Step: 8, Loss: 0.2844295799732208
Step: 9, Loss: 0.30456510186195374
Step: 10, Loss: 0.2801378667354584
Step: 11, Loss: 0.29828131198883057
Step: 12, Loss: 0.2962247133255005
Step: 13, Loss: 0.30923017859458923
Step: 14, Loss: 0.31091827154159546
Step: 15, Loss: 0.27789679169654846
Step: 16, Loss: 0.2624688446521759
Step: 17, Loss: 0.22511954605579376
Step: 18, Loss: 0.280689001083374
Step: 19, Loss: 0.27189648151397705
Step: 20, Loss: 0.24390874803066254
Step: 21, Loss: 0.2661014497280121
Step: 22, Loss: 0.29457494616508484
Step: 23, Loss: 0.25290507078170776
Step: 24, Loss: 0.3124399483203888
Step: 25, Loss: 0.23791244626045227
Step: 26, Loss: 0.28939586877822876
Step: 27, Loss: 0.3050451874732971
Step: 28, Loss: 0.23892009258270264
Step: 29, Loss: 0

Step: 229, Loss: 0.27217331528663635
Step: 230, Loss: 0.3318787217140198
Step: 231, Loss: 0.26237544417381287
Step: 232, Loss: 0.27074697613716125
Step: 233, Loss: 0.27756467461586
Step: 234, Loss: 0.2709818482398987
Step: 235, Loss: 0.28040847182273865
Step: 236, Loss: 0.2440173178911209
Step: 237, Loss: 0.26873642206192017
Step: 238, Loss: 0.2682378590106964
Step: 239, Loss: 0.298973947763443
Step: 240, Loss: 0.3035852313041687
Step: 241, Loss: 0.2427515834569931
Step: 242, Loss: 0.26286453008651733
Step: 243, Loss: 0.24577447772026062
Step: 244, Loss: 0.2848333716392517
Step: 245, Loss: 0.25182920694351196
Step: 246, Loss: 0.30272576212882996
Step: 247, Loss: 0.21794329583644867
Step: 248, Loss: 0.2515225410461426
Step: 249, Loss: 0.24018867313861847
Step: 250, Loss: 0.24686479568481445
Step: 251, Loss: 0.30571597814559937
Step: 252, Loss: 0.29297637939453125
Step: 253, Loss: 0.2636632025241852
Step: 254, Loss: 0.2841168940067291
Step: 255, Loss: 0.26971930265426636
Step: 256, Loss:

In [ ]:
test_data_loader = T.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=True)
testload = iter(test_data_loader)
testlen = len(testload)

In [ ]:
correct = 0
diffy.eval()
#embedder.eval()
for i in range(testlen):
    with T.no_grad():
        ins, tru = next(testload)
        
        #embed = embedder(ins.reshape(batch_size, 2000).to(T.long))
        output, (c, m, r) = diffy(ins, (None, memory, None), reset_experience=False)
        m = { k : (v.detach() if isinstance(v, T.autograd.Variable) else v) for k, v in m.items()}
        (c, m, r) = (None, None, None)
        #output_2 = output[:,-1]
        y_pred = T.sum(output, (1,2), keepdim=True).round()
        
        for b in range(batch_size):
            if(y_pred[b] == tru[b]):
                correct = correct+1
                
        print('Step: {}, Accuracy: {}'.format(i+1, correct/((i+1)*batch_size)) )
            
print('Final Accuracy: {}'.format(correct/(testlen * batch_size)))

## Save the model and memory

In [ ]:
file = open('diffy-sentiment', 'wb')
pickle.dump(diffy, file)
file.close()

In [ ]:
file = open('memory-sentiment', 'wb')
pickle.dump(memory, file)
file.close()

In [ ]:
file = open('embed-sentiment', 'wb')
pickle.dump(embedder, file)
file.close()

In [ ]:
file = open('diffy-sentiment', 'wb')
diffy = pickle.load(file)
file.close()

file = open('memory-sentiment', 'wb')
memory = pickle.load(file)
file.close()

file = open('embed-sentiment', 'wb')
embedder = pickle.load(file)
file.close()